# Задание по программированию: Рекомендательные системы

## Описание задачи

Небольшой интернет-магазин попросил вас добавить ранжирование товаров в блок "Смотрели ранее" - в нем теперь надо показывать не последние просмотренные пользователем товары, а те товары из просмотренных, которые он наиболее вероятно купит. Качество вашего решения будет оцениваться по количеству покупок в сравнении с прошлым решением в ходе А/В теста, т.к. по доходу от продаж статзначимость будет достигаться дольше из-за разброса цен. Таким образом, ничего заранее не зная про корреляцию оффлайновых и онлайновых метрик качества, в начале проекта вы можете лишь постараться оптимизировать recall@k и precision@k.

Это задание посвящено построению простых бейзлайнов для этой задачи: ранжирование просмотренных товаров по частоте просмотров и по частоте покупок. Эти бейзлайны, с одной стороны, могут помочь вам грубо оценить возможный эффект от ранжирования товаров в блоке - например, чтобы вписать какие-то числа в коммерческое предложение заказчику, а с другой стороны, могут оказаться самым хорошим вариантом, если данных очень мало (недостаточно для обучения даже простых моделей).

Входные данные

Вам дается две выборки с пользовательскими сессиями - id-шниками просмотренных и id-шниками купленных товаров. Одна выборка будет использоваться для обучения (оценки популярностей товаров), а другая - для теста.

В файлах записаны сессии по одной в каждой строке. Формат сессии: id просмотренных товаров через , затем идёт ; после чего следуют id купленных товаров (если такие имеются), разделённые запятой. Например, 1,2,3,4; или 1,2,3,4;5,6.

Гарантируется, что среди id купленных товаров все различные.

In [1]:
import pandas as pd
import numpy as np

In [10]:
train = pd.read_csv('coursera_sessions_train.txt', sep=';', header=None)
train.columns = ['Seen', 'Bought']

In [11]:
test = pd.read_csv('coursera_sessions_test.txt', delimiter=';', header=None)
test.columns = ['Seen', 'Bought']

In [4]:
test.head()

,Seen,Bought
0,"6,7,8",NaN
1,"13,14,15",NaN
2,"22,23",NaN
3,"28,29,30,31,32,33",NaN
4,"40,41",NaN


In [5]:
train.head()

,Seen,Bought
0,"0,1,2,3,4,5",NaN
1,"9,10,11,9,11,12,9,11",NaN
2,"16,17,18,19,20,21",NaN
3,"24,25,26,27,24",NaN
4,"34,35,36,34,37,35,36,37,38,39,38,39",NaN



Важно:

Сессии, в которых пользователь ничего не купил, исключаем из оценки качества.
Если товар не встречался в обучающей выборке, его популярность равна 0.
Рекомендуем разные товары. И их число должно быть не больше, чем количество различных просмотренных пользователем товаров.
Рекомендаций всегда не больше, чем минимум из двух чисел: количество просмотренных пользователем товаров и k в recall@k / precision@k.

In [12]:
train.dropna(inplace=True)

## Задание

1. На обучении постройте частоты появления id в просмотренных и в купленных (id может несколько раз появляться в просмотренных, все появления надо учитывать)
2. Реализуйте два алгоритма рекомендаций:
    - сортировка просмотренных id по популярности (частота появления в просмотренных),
    - сортировка просмотренных id по покупаемости (частота появления в покупках).
3. Для данных алгоритмов выпишите через пробел AverageRecall@1, AveragePrecision@1, AverageRecall@5, AveragePrecision@5 на обучающей и тестовых выборках, округляя до 2 знака после запятой. Это будут ваши ответы в этом задании. Посмотрите, как они соотносятся друг с другом. Где качество получилось выше? Значимо ли это различие? Обратите внимание на различие качества на обучающей и тестовой выборке в случае рекомендаций по частотам покупки.

Если частота одинаковая, то сортировать нужно по возрастанию момента просмотра (чем раньше появился в просмотренных, тем больше приоритет)

In [6]:
seen_frequences = pd.Series(','.join(train.Seen.values).split(',')).value_counts()
bought_frequences = pd.Series(','.join(train.Bought.dropna().values).split(',')).value_counts()

In [13]:
train['seen_arr'] = train.apply(lambda x: list(seen_frequences[pd.unique(x.Seen.split(','))].sort_values(ascending=False, kind='mergesort').index), axis=1)
train.head()

,Seen,Bought,seen_arr
7,"59,60,61,62,60,63,64,65,66,61,67,68,67","67,60,63","[63, 64, 68, 67, 66, 65, 61, 60, 62, 59]"
10,"84,85,86,87,88,89,84,90,91,92,93,86",86,"[85, 93, 89, 90, 92, 84, 86, 87, 91, 88]"
19,"138,198,199,127",199,"[127, 138, 198, 199]"
30,"303,304,305,306,307,308,309,310,311,312",303,"[303, 306, 310, 309, 307, 304, 312, 311, 308, ..."
33,"352,353,352",352,"[352, 353]"


In [14]:
train['Recall_1'] = train.apply(lambda x: len(seen_frequences[pd.unique(x.Seen.split(','))].sort_values(ascending=False, kind='mergesort')[:1].reindex(x.Bought.split(',')).dropna())/len(x.Bought.split(',')), axis=1)
train['Precision_1'] = train.apply(lambda x: len(seen_frequences[pd.unique(x.Seen.split(','))].sort_values(ascending=False, kind='mergesort')[:1].reindex(x.Bought.split(',')).dropna())/1, axis=1)
train['Recall_5'] = train.apply(lambda x: len(seen_frequences[pd.unique(x.Seen.split(','))].sort_values(ascending=False, kind='mergesort')[:5].reindex(x.Bought.split(',')).dropna())/len(x.Bought.split(',')), axis=1)
train['Precision_5'] = train.apply(lambda x: len(seen_frequences[pd.unique(x.Seen.split(','))].sort_values(ascending=False, kind='mergesort')[:5].reindex(x.Bought.split(',')).dropna())/5, axis=1)
train.head()

,Seen,Bought,seen_arr,Recall_1,Precision_1,Recall_5,Precision_5
7,"59,60,61,62,60,63,64,65,66,61,67,68,67","67,60,63","[63, 64, 68, 67, 66, 65, 61, 60, 62, 59]",0.333333,1.0,0.666667,0.4
10,"84,85,86,87,88,89,84,90,91,92,93,86",86,"[85, 93, 89, 90, 92, 84, 86, 87, 91, 88]",0.000000,0.0,0.000000,0.0
19,"138,198,199,127",199,"[127, 138, 198, 199]",0.000000,0.0,1.000000,0.2
30,"303,304,305,306,307,308,309,310,311,312",303,"[303, 306, 310, 309, 307, 304, 312, 311, 308, ...",1.000000,1.0,1.000000,0.2
33,"352,353,352",352,"[352, 353]",1.000000,1.0,1.000000,0.2


In [15]:
train['bought_arr'] = train.apply(lambda x: list(bought_frequences[pd.unique(x.Seen.split(','))].fillna(0).sort_values(ascending=False, kind='mergesort').index), axis=1)

In [16]:
train.drop(['Precision_5', 'Precision_1'], axis=1).head()

,Seen,Bought,seen_arr,Recall_1,Recall_5,bought_arr
7,"59,60,61,62,60,63,64,65,66,61,67,68,67","67,60,63","[63, 64, 68, 67, 66, 65, 61, 60, 62, 59]",0.333333,0.666667,"[67, 63, 60, 68, 66, 65, 64, 62, 61, 59]"
10,"84,85,86,87,88,89,84,90,91,92,93,86",86,"[85, 93, 89, 90, 92, 84, 86, 87, 91, 88]",0.000000,0.000000,"[86, 93, 85, 92, 91, 90, 89, 88, 87, 84]"
19,"138,198,199,127",199,"[127, 138, 198, 199]",0.000000,1.000000,"[127, 199, 138, 198]"
30,"303,304,305,306,307,308,309,310,311,312",303,"[303, 306, 310, 309, 307, 304, 312, 311, 308, ...",1.000000,1.000000,"[303, 312, 311, 310, 309, 308, 307, 306, 305, ..."
33,"352,353,352",352,"[352, 353]",1.000000,1.000000,"[352, 353]"


In [17]:
train['b_Recall_1'] = train.apply(lambda x: len(pd.DataFrame(bought_frequences[pd.unique(x.Seen.split(','))].fillna(0)).sort_values(by=0, ascending=False, kind='mergesort')[:1].reindex(x.Bought.split(',')).dropna())/len(x.Bought.split(',')), axis=1)
train['b_Precision_1'] = train.apply(lambda x: len(pd.DataFrame(bought_frequences[pd.unique(x.Seen.split(','))].fillna(0)).sort_values(by=0, ascending=False, kind='mergesort')[:1].reindex(x.Bought.split(',')).dropna())/1, axis=1)
train['b_Recall_5'] = train.apply(lambda x: len(pd.DataFrame(bought_frequences[pd.unique(x.Seen.split(','))].fillna(0)).sort_values(by=0, ascending=False, kind='mergesort')[:5].reindex(x.Bought.split(',')).dropna())/len(x.Bought.split(',')), axis=1)
train['b_Precision_5'] = train.apply(lambda x: len(pd.DataFrame(bought_frequences[pd.unique(x.Seen.split(','))].fillna(0)).sort_values(by=0, ascending=False, kind='mergesort')[:5].reindex(x.Bought.split(',')).dropna())/5, axis=1)
train.head()

,Seen,Bought,seen_arr,Recall_1,Precision_1,Recall_5,Precision_5,bought_arr,b_Recall_1,b_Precision_1,b_Recall_5,b_Precision_5
7,"59,60,61,62,60,63,64,65,66,61,67,68,67","67,60,63","[63, 64, 68, 67, 66, 65, 61, 60, 62, 59]",0.333333,1.0,0.666667,0.4,"[67, 63, 60, 68, 66, 65, 64, 62, 61, 59]",0.333333,1.0,1.0,0.6
10,"84,85,86,87,88,89,84,90,91,92,93,86",86,"[85, 93, 89, 90, 92, 84, 86, 87, 91, 88]",0.000000,0.0,0.000000,0.0,"[86, 93, 85, 92, 91, 90, 89, 88, 87, 84]",1.000000,1.0,1.0,0.2
19,"138,198,199,127",199,"[127, 138, 198, 199]",0.000000,0.0,1.000000,0.2,"[127, 199, 138, 198]",0.000000,0.0,1.0,0.2
30,"303,304,305,306,307,308,309,310,311,312",303,"[303, 306, 310, 309, 307, 304, 312, 311, 308, ...",1.000000,1.0,1.000000,0.2,"[303, 312, 311, 310, 309, 308, 307, 306, 305, ...",1.000000,1.0,1.0,0.2
33,"352,353,352",352,"[352, 353]",1.000000,1.0,1.000000,0.2,"[352, 353]",1.000000,1.0,1.0,0.2


In [274]:
train.describe()

,Precision_5,Precision_1,Recall_5,Recall_1,b_Recall_1,b_Precision_1,b_Recall_5,b_Precision_5
count,3608.000000,3608.000000,3608.000000,3608.000000,3608.000000,3608.000000,3608.000000,3608.00000
mean,0.212084,0.504989,0.823229,0.436872,0.688449,0.803769,0.926307,0.25255
std,0.115494,0.500044,0.339979,0.469362,0.413695,0.397200,0.214355,0.13734
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
25%,0.200000,0.000000,1.000000,0.000000,0.333333,1.000000,1.000000,0.20000
50%,0.200000,1.000000,1.000000,0.154762,1.000000,1.000000,1.000000,0.20000
75%,0.200000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.20000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000


In [18]:
test.dropna(inplace=True)

In [20]:
test['Recall_1'] = test.apply(lambda x: len(seen_frequences[pd.unique(x.Seen.split(','))].fillna(0).sort_values(ascending=False, kind='mergesort')[:1].reindex(x.Bought.split(',')).dropna())/len(x.Bought.split(',')), axis=1)
test['Precision_1'] = test.apply(lambda x: len(seen_frequences[pd.unique(x.Seen.split(','))].fillna(0).sort_values(ascending=False, kind='mergesort')[:1].reindex(x.Bought.split(',')).dropna())/1, axis=1)
test['Recall_5'] = test.apply(lambda x: len(seen_frequences[pd.unique(x.Seen.split(','))].fillna(0).sort_values(ascending=False, kind='mergesort')[:5].reindex(x.Bought.split(',')).dropna())/len(x.Bought.split(',')), axis=1)
test['Precision_5'] = test.apply(lambda x: len(seen_frequences[pd.unique(x.Seen.split(','))].fillna(0).sort_values(ascending=False, kind='mergesort')[:5].reindex(x.Bought.split(',')).dropna())/5, axis=1)
test.head()

,Seen,Bought,Recall_1,Precision_1,Recall_5,Precision_5
7,"63,68,69,70,66,61,59,61,66,68","66,63",0.5,1.0,1.0,0.4
14,"158,159,160,159,161,162",162,0.0,0.0,1.0,0.2
19,"200,201,202,203,204","201,205",0.0,0.0,0.5,0.2
34,"371,372,371","371,373",0.5,1.0,0.5,0.2
40,422,422,1.0,1.0,1.0,0.2


In [21]:
test['b_Recall_1'] = test.apply(lambda x: len(pd.DataFrame(bought_frequences[pd.unique(x.Seen.split(','))].fillna(0)).sort_values(by=0, ascending=False, kind='mergesort')[:1].reindex(x.Bought.split(',')).dropna())/len(x.Bought.split(',')), axis=1)
test['b_Precision_1'] = test.apply(lambda x: len(pd.DataFrame(bought_frequences[pd.unique(x.Seen.split(','))].fillna(0)).sort_values(by=0, ascending=False, kind='mergesort')[:1].reindex(x.Bought.split(',')).dropna())/1, axis=1)
test['b_Recall_5'] = test.apply(lambda x: len(pd.DataFrame(bought_frequences[pd.unique(x.Seen.split(','))].fillna(0)).sort_values(by=0, ascending=False, kind='mergesort')[:5].reindex(x.Bought.split(',')).dropna())/len(x.Bought.split(',')), axis=1)
test['b_Precision_5'] = test.apply(lambda x: len(pd.DataFrame(bought_frequences[pd.unique(x.Seen.split(','))].fillna(0)).sort_values(by=0, ascending=False, kind='mergesort')[:5].reindex(x.Bought.split(',')).dropna())/5, axis=1)
test.tail()

,Seen,Bought,Recall_1,Precision_1,Recall_5,Precision_5,b_Recall_1,b_Precision_1,b_Recall_5,b_Precision_5
49943,"2859,2854,88887,96997,4439,28645,99975,34601,1...","24907,102691,18496,4333",0.0,0.0,0.5,0.4,0.0,0.0,0.25,0.2
49944,"77655,23249,1306,47450,26157,58205,47450,58205...","58205,3111,69482",0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
49945,"60538,44430,66252,44430,60538,66251","66252,44430",0.0,0.0,1.0,0.4,0.0,0.0,1.00,0.4
49946,"49815,76363",49815,1.0,1.0,1.0,0.2,1.0,1.0,1.00,0.2
49985,"21841,17711,21841,17711,21841,17711,21841,1771...",21841,0.0,0.0,1.0,0.2,1.0,1.0,1.00,0.2


In [25]:
test.describe()

,Recall_1,Precision_1,Recall_5,Precision_5,b_Recall_1,b_Precision_1,b_Recall_5,b_Precision_5
count,3665.000000,3665.000000,3665.000000,3665.000000,3665.000000,3665.000000,3665.000000,3665.000000
mean,0.410250,0.473397,0.796578,0.203492,0.460620,0.527694,0.820187,0.210095
std,0.466512,0.499360,0.363913,0.116391,0.471753,0.499301,0.342072,0.113771
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.666667,0.200000,0.000000,0.000000,1.000000,0.200000
50%,0.000000,0.000000,1.000000,0.200000,0.333333,1.000000,1.000000,0.200000
75%,1.000000,1.000000,1.000000,0.200000,1.000000,1.000000,1.000000,0.200000
max,1.000000,1.000000,1.000000,0.800000,1.000000,1.000000,1.000000,0.800000


In [26]:
r=test.describe()

In [67]:
def write_to_submission_file(answer, out_file):
    with open(out_file, 'w') as f:
        if isinstance(answer, list):
            f.write(' '.join(answer))
        else:
            f.write(str(answer))
        
    print('Overwriting {}'.format(out_file))

Рекомендации по частоте просмотров товаров - качество на обучающей выборке:

In [63]:
ans1 = train[['Recall_1','Precision_1','Recall_5', 'Precision_5']].mean()
write_to_submission_file(list(map(lambda x: str(round(x,2)), list(ans1))), 'PA_answer1.txt')
print(ans1)

Overwriting "PA_answer1.txt"
Recall_1       0.436872
Precision_1    0.504989
Recall_5       0.823229
Precision_5    0.212084
dtype: float64


Рекомендации по частоте просмотров товаров - качество на тестовой выборке:

In [68]:
ans2=test[['Recall_1','Precision_1','Recall_5', 'Precision_5']].mean()
write_to_submission_file(list(map(lambda x: str(round(x,2)), list(ans2))), 'PA_answer2.txt')
print(ans2)

Overwriting PA_answer2.txt
Recall_1       0.410250
Precision_1    0.473397
Recall_5       0.796578
Precision_5    0.203492
dtype: float64


Рекомендации по частоте покупок товаров - качество на обучающей выборке:

In [69]:
ans3=train[['b_Recall_1','b_Precision_1','b_Recall_5', 'b_Precision_5']].mean()
write_to_submission_file(list(map(lambda x: str(round(x,2)), list(ans3))), 'PA_answer3.txt')
print(ans3)

Overwriting PA_answer3.txt
b_Recall_1       0.688449
b_Precision_1    0.803769
b_Recall_5       0.926307
b_Precision_5    0.252550
dtype: float64


Рекомендации по частоте покупок товаров - качество на тестовой выборке:

In [70]:
ans4=test[['b_Recall_1','b_Precision_1','b_Recall_5', 'b_Precision_5']].mean()
write_to_submission_file(list(map(lambda x: str(round(x,2)), list(ans4))), 'PA_answer4.txt')
print(ans4)

Overwriting PA_answer4.txt
b_Recall_1       0.460620
b_Precision_1    0.527694
b_Recall_5       0.820187
b_Precision_5    0.210095
dtype: float64
